In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import numpy as np
import os
import json
import matplotlib.pyplot as plt

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,albany,US,2022-07-21 23:30:46,42.6001,-73.9662,90.01,48,17,8.99
1,1,vostok,RU,2022-07-21 23:30:25,46.4856,135.8833,62.08,97,99,1.39
2,2,ushuaia,AR,2022-07-21 23:30:50,-54.8000,-68.3000,31.66,86,75,28.77
3,3,tiznit,MA,2022-07-21 23:30:50,29.5833,-9.5000,88.20,21,22,3.09
4,4,te anau,NZ,2022-07-21 23:28:15,-45.4167,167.7167,39.22,91,67,1.21


In [3]:
city_data_df.dtypes

Unnamed: 0      int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of wind
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum wind.
wind = city_data_df["Wind Speed"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,tiznit,MA,2022-07-21 23:30:50,29.5833,-9.5000,88.20,21,22,3.09
5,5,lagos,NG,2022-07-21 23:30:08,6.5833,3.7500,75.49,84,98,5.44
6,6,butaritari,KI,2022-07-21 23:29:34,3.0707,172.7902,81.48,73,57,5.41
7,7,lompoc,US,2022-07-21 23:30:52,34.6391,-120.4579,88.02,72,0,17.00
12,12,wallaceburg,CA,2022-07-21 23:30:53,42.5930,-82.3885,89.11,50,2,13.67
16,16,manado,ID,2022-07-21 23:30:54,1.4870,124.8455,79.57,83,20,3.44
20,20,avarua,CK,2022-07-21 23:29:13,-21.2078,-159.7750,75.25,73,75,16.11
21,21,acarau,BR,2022-07-21 23:26:49,-2.8856,-40.1200,78.82,78,49,13.78
22,22,mecca,SA,2022-07-21 23:30:57,21.4267,39.8261,85.08,56,50,3.04
27,27,dong hoi,VN,2022-07-21 23:30:59,17.4833,106.6000,79.14,79,84,2.68


In [11]:
preferred_cities_df.count()

Unnamed: 0    210
City          210
Country       210
Date          210
Lat           210
Lng           210
Max Temp      210
Humidity      210
Cloudiness    210
Wind Speed    210
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,tiznit,MA,88.20,29.5833,-9.5000,
5,lagos,NG,75.49,6.5833,3.7500,
6,butaritari,KI,81.48,3.0707,172.7902,
7,lompoc,US,88.02,34.6391,-120.4579,
12,wallaceburg,CA,89.11,42.5930,-82.3885,
16,manado,ID,79.57,1.4870,124.8455,
20,avarua,CK,75.25,-21.2078,-159.7750,
21,acarau,BR,78.82,-2.8856,-40.1200,
22,mecca,SA,85.08,21.4267,39.8261,
27,dong hoi,VN,79.14,17.4833,106.6000,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params = params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    results = response["results"]
    # Grab the first hotel from the results and store the name.
    try:
        print(f"Clostest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
    
    print("--------")
    

Retrieving Results for Index 3: tiznit.
Hotel not found... skipping.
--------
Retrieving Results for Index 5: lagos.
Hotel not found... skipping.
--------
Retrieving Results for Index 6: butaritari.
Clostest hotel is Isles Sunset Lodge.
--------
Retrieving Results for Index 7: lompoc.
Clostest hotel is Holiday Inn Express Lompoc, an IHG Hotel.
--------
Retrieving Results for Index 12: wallaceburg.
Clostest hotel is Wallaceburg Inn.
--------
Retrieving Results for Index 16: manado.
Clostest hotel is Aryaduta Hotel.
--------
Retrieving Results for Index 20: avarua.
Clostest hotel is Paradise Inn.
--------
Retrieving Results for Index 21: acarau.
Clostest hotel is castelo pizzaria.
--------
Retrieving Results for Index 22: mecca.
Clostest hotel is Makkah Clock Royal Tower, A Fairmont Hotel.
--------
Retrieving Results for Index 27: dong hoi.
Clostest hotel is Nam Long Hotel.
--------
Retrieving Results for Index 30: honiara.
Clostest hotel is Heritage Park Hotel.
--------
Retrieving Resul

Clostest hotel is Savannah Inn.
--------
Retrieving Results for Index 271: cairns.
Clostest hotel is Mantra Trilogy Cairns.
--------
Retrieving Results for Index 272: bethel.
Clostest hotel is Courtyard by Marriott Danbury.
--------
Retrieving Results for Index 273: samarai.
Clostest hotel is Nuli Sapi.
--------
Retrieving Results for Index 275: carutapera.
Clostest hotel is Lidera.
--------
Retrieving Results for Index 280: celestun.
Clostest hotel is Posada Lilia.
--------
Retrieving Results for Index 285: skeldon.
Clostest hotel is Classic International Hotel & Suites.
--------
Retrieving Results for Index 286: porbandar.
Clostest hotel is Lords Inn Porbandar.
--------
Retrieving Results for Index 287: hinton.
Clostest hotel is Econo Lodge & Suites.
--------
Retrieving Results for Index 288: katobu.
Clostest hotel is La Ode Husein.
--------
Retrieving Results for Index 290: balkanabat.
Clostest hotel is Nebitchi Hotel.
--------
Retrieving Results for Index 291: kieta.
Clostest hotel

Clostest hotel is Anchor Inn & Suites.
--------
Retrieving Results for Index 497: amuntai.
Clostest hotel is Hotel Balqis.
--------
Retrieving Results for Index 498: villafranca de los barros.
Clostest hotel is HOTEL ACOSTA CIUDAD DE LA MUSICA.
--------
Retrieving Results for Index 504: robertsport.
Clostest hotel is Kwepunha Retreat.
--------
Retrieving Results for Index 505: sur.
Clostest hotel is Sur Plaza Hotel.
--------
Retrieving Results for Index 512: the valley.
Clostest hotel is CeBlue Villas & Beach Resort.
--------
Retrieving Results for Index 515: iwanai.
Clostest hotel is いわない高原ホテル.
--------
Retrieving Results for Index 516: gizo.
Clostest hotel is Imagination Island.
--------
Retrieving Results for Index 521: brookings.
Clostest hotel is Holiday Inn Express & Suites Brookings, an IHG Hotel.
--------
Retrieving Results for Index 524: cayenne.
Clostest hotel is Hôtel Le Dronmi.
--------
Retrieving Results for Index 526: zhuanghe.
Clostest hotel is Hanting Express Dalian Zhu

In [14]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,tiznit,MA,88.20,29.5833,-9.5000,
5,lagos,NG,75.49,6.5833,3.7500,
6,butaritari,KI,81.48,3.0707,172.7902,Isles Sunset Lodge
7,lompoc,US,88.02,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
12,wallaceburg,CA,89.11,42.5930,-82.3885,Wallaceburg Inn
...,...,...,...,...,...,...
564,sola,VU,80.26,-13.8833,167.5500,Leumerus Bungalows
568,kupang,ID,78.78,-10.1667,123.5833,Hotel On The Rock by Prasanthi
572,umm kaddadah,SD,78.28,13.6017,26.6876,
573,mangrol,IN,81.45,21.1167,70.1167,Hotel The Royal Honours


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,dissipating = False,
             max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))